In [4]:
from yerbamate import env

from twitter.data.loader.spark import get_tweets_session, spark
import os
from pyspark.sql.functions import col, explode, count, from_json

from pyspark.sql.types import ArrayType, StringType
# env = yerbamate.Environment()



time_filter = ((col("date") < "2023-04-17"))


columns = ["userId", "hashtags", "date"]

parquet_df = get_tweets_session(columns)

# this is spark dataframe
# drop null hashtags
parquet_df.na.drop(subset=["hashtags"])

# date after 2022
parquet_df = parquet_df.filter(
    time_filter
)

label_path = os.path.join(env["save"], "hashtag_propaganda_labeled.csv")
# read CSV file into DataFrame
propaganda_labels_df = spark.read.csv(label_path, header=True)


# Define the schema for the 'affiliation' column
array_schema = ArrayType(StringType())

# Convert the 'affiliation' column from a JSON string to an array
propaganda_labels_df = propaganda_labels_df.withColumn(
    "affiliation", 
    from_json("affiliation", array_schema)
)
propaganda_labels_df = propaganda_labels_df.select("hashtag", explode(col("affiliation")).alias("affiliation"))

parquet_df = parquet_df.select("userId", explode(col("hashtags")).alias("hashtag"))

parquet_df = parquet_df.drop("hashtags")


joined_df = parquet_df.join(propaganda_labels_df, "hashtag", "inner")

usage_count = joined_df.groupBy('userId', 'affiliation').agg(count('hashtag').alias('count'))

usage_count.show()

+------------------+----------------+-----+
|            userId|     affiliation|count|
+------------------+----------------+-----+
|        1930624009|            Shia|   25|
|         408795883|            Shia|    6|
|         143531181|Islamic Republic|   47|
|        2473674486|            Shia|   29|
|         153841889|            Shia|   15|
|         308068500|            Left|    1|
|         281961592|            Shia|   26|
|         144105935|Islamic Republic|   33|
|          15054135|            Shia|    8|
|         557746834|Islamic Republic|    1|
|         351120282|Islamic Republic|   30|
|         243529205|            Shia|    7|
|          24744541|             PKK|    3|
|         319522800|            Shia|    4|
|        1735011630|        Monarchy|    4|
|         216289357|            Shia|   16|
|898593847059632128|             MEK|    3|
|898593847059632128|Islamic Republic|    1|
|        2723733757|           Right|    2|
|         711881270|Islamic Repu

In [5]:
pandas_df =  usage_count.toPandas()

ERROR:root:KeyboardInterrupt while sending command.====>   (14544 + 16) / 15681]
Traceback (most recent call last):
  File "/home/al/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/al/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/anaconda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 